<a href="https://colab.research.google.com/github/FurqanBhat/LSTM-TEXT-GENERATOR/blob/main/LSTM_%26_Text_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [85]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size

    self.forget_gate = nn.Linear(input_size + hidden_size, hidden_size)
    self.input_gate = nn.Linear(input_size + hidden_size, hidden_size)
    self.cell_gate = nn.Linear(input_size + hidden_size, hidden_size)
    self.output_gate = nn.Linear(input_size + hidden_size, hidden_size)

  def forward(self, input, hidden, cell):
    combined = torch.cat((input, hidden), 1)

    forget_gate_output = torch.sigmoid(self.forget_gate(combined))
    input_gate_output = torch.sigmoid(self.input_gate(combined))
    cell_gate_output = torch.tanh(self.cell_gate(combined))
    output_gate_output = torch.sigmoid(self.output_gate(combined))

    cell = forget_gate_output * cell + input_gate_output * cell_gate_output
    hidden = output_gate_output * torch.tanh(cell)

    return hidden, cell

In [86]:
device=torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [87]:
class NEWLSTM(nn.Module):
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size

    self.forget_gate = nn.Linear(input_size + hidden_size, hidden_size)
    # self.input_gate = nn.Linear(input_size + hidden_size, hidden_size)
    self.cell_gate = nn.Linear(input_size + hidden_size, hidden_size)
    self.output_gate = nn.Linear(input_size + hidden_size, hidden_size)

  def forward(self, input, hidden, cell):
    combined = torch.cat((input, hidden), 1)

    forget_gate_output = torch.sigmoid(self.forget_gate(combined))
    input_gate_output = 1-forget_gate_output
    cell_gate_output = torch.tanh(self.cell_gate(combined))
    output_gate_output = torch.sigmoid(self.output_gate(combined))

    cell = forget_gate_output * cell + input_gate_output * cell_gate_output
    hidden = output_gate_output * torch.tanh(cell)

    return hidden, cell

In [88]:
#data loading
with open("shakespeare_100.txt", "r") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

char2idx = {ch: i for i, ch in enumerate(chars)}
idx2char = {i: ch for ch, i in char2idx.items()}
encoded_text = [char2idx[c] for c in text]



In [89]:
#seq preparation
seq_length = 100
inputs, targets = [], []
for i in range(0, len(encoded_text) - seq_length):
    inputs.append(encoded_text[i:i+seq_length])
    targets.append(encoded_text[i+1:i+seq_length+1])

input_tensor = torch.tensor(inputs, dtype=torch.long)
target_tensor = torch.tensor(targets, dtype=torch.long)

dataset = TensorDataset(input_tensor, target_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


In [90]:
class ShakespeareGenerator(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, input_size)
        self.lstm = NEWLSTM(input_size, hidden_size)  # custom  LSTM
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h, c):
        batch_size, seq_len = x.shape
        outputs = []

        for t in range(seq_len):
            embedded = self.embedding(x[:, t])
            h, c = self.lstm(embedded, h, c)
            logits = self.fc(h)
            outputs.append(logits)

        return torch.stack(outputs, dim=1), h, c

In [91]:
# === Hyperparameters & Setup ===
input_size = 128
hidden_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ShakespeareGenerator(vocab_size, input_size, hidden_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
loss_fn = nn.CrossEntropyLoss()


In [92]:
import time

# === Training Loop ===
start_time = time.time()  # Record the start time

for epoch in range(10):
    model.train()
    total_loss = 0
    for x_batch, y_batch in dataloader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        h = torch.zeros(x_batch.size(0), hidden_size).to(device)
        c = torch.zeros(x_batch.size(0), hidden_size).to(device)

        optimizer.zero_grad()
        output, _, _ = model(x_batch, h, c)
        loss = loss_fn(output.view(-1, vocab_size), y_batch.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader):.4f}")

end_time = time.time()  # Record the end time
training_time = end_time - start_time  # Calculate training time

print(f"Training Time: {training_time:.2f} seconds")

Epoch 1, Loss: 2.6741
Epoch 2, Loss: 1.7427
Epoch 3, Loss: 0.9963
Epoch 4, Loss: 0.4004
Epoch 5, Loss: 0.1686
Epoch 6, Loss: 0.1142
Epoch 7, Loss: 0.0946
Epoch 8, Loss: 0.0842
Epoch 9, Loss: 0.0779
Epoch 10, Loss: 0.0743
Training Time: 207.55 seconds


In [93]:
# torch.save(model.state_dict(), './model.pth') #

In [97]:
def generate(model, start_text, char2idx, idx2char, length=100, device='cpu'):
    model.eval()  # Set the model to evaluation mode
    # Create input tensor and move it to the specified device
    input = torch.tensor([[char2idx[c] for c in start_text]], device=device)
    # Create hidden and cell state tensors and move them to the specified device
    h = torch.zeros(1, model.lstm.hidden_size, device=device)
    c = torch.zeros(1, model.lstm.hidden_size, device=device)

    result = start_text

    for _ in range(length):
        out, h, c = model(input, h, c)
        last_logits = out[0, -1]
        probs = torch.softmax(last_logits, dim=0)
        next_char_idx = torch.multinomial(probs, 1).item()
        result += idx2char[next_char_idx]
        input = torch.tensor([[next_char_idx]], device=device)

    return result

In [99]:
print(generate(model, "End", char2idx, idx2char, length=500, device=device))

End unkind of post.

AJAX:
O Mattle! these strict cuttances cut
it down; for I had slept my fellow, to said 'I
Have found your lordship hope.

MARK ANTONY:
How! I have my hanking
that he contends, he hath restraints would not then be got as he prayers
Which are much thus from them with thee; I am not ever thought
To make that with a wise exclaim, as I am sure;
To say well like a dotage on the fixed cease,
And let mine eyes may straight sole sword conveyard,
That dust
it down; for I had slept my fel
